# Fusion ResNet Training on Google Colab

This notebook allows you to run the Fusion ResNet training pipeline on Google Colab using a free GPU.

### Instructions:
1.  **Upload Data & Code**: Upload your entire `Research_Project` folder (containing `Code` and `Datasets`) to your Google Drive.
2.  **Mount Drive**: Run the cell below to mount your Google Drive.
3.  **Install Dependencies**: Run the installation cell.
4.  **Data Augmentation**: Run `data_ingestion.py` which now parses 20-min overlapping windows (5x Dataset Expansion).
5.  **Run Training**: Scalable Training with `AdamW` optimizer.
6.  **Run XAI**: Generate explainability plots.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import sys

# --- CONFIGURATION ---
# UPDATE THIS PATH to where you uploaded the 'Research Project' folder
# Example: If you uploaded 'Research Project' to the root of 'MyDrive'
PROJECT_ROOT = "/content/drive/MyDrive/Research_Project/"
CODE_DIR = os.path.join(PROJECT_ROOT, "Code")

# Check if path exists
if not os.path.exists(PROJECT_ROOT):
    print(f"ERROR: Path not found: {PROJECT_ROOT}")
    print("Please verify your folder structure on Google Drive.")
else:
    print(f"Project found at: {PROJECT_ROOT}")
    os.chdir(CODE_DIR)
    sys.path.append(CODE_DIR)
    print(f"Working directory set to: {os.getcwd()}")

### Install Dependencies

In [ ]:
!pip install wfdb pandas==2.2.3 shap

### Step 1: Data Ingestion (Optional if already run locally)
If you uploaded the `Datasets/processed` folder, you can skip this. If not, run this cell.

In [ ]:
!python data_ingestion.py

### Step 2: Training
This uses GPU if available.

In [ ]:
!python train.py

### Step 3: Explainability (XAI)
Generates Grad-CAM and SHAP plots in `Code/figures/`.

In [ ]:
!python xai.py

In [ ]:
import tensorflow as tf
import os

def quantize_model(fold_number):
    # 1. Load the trained Keras model
    model_path = f'/content/drive/MyDrive/Research_Project/Code/models/best_model_fold_{fold_number}.keras'
    model = tf.keras.models.load_model(model_path)

    # 2. Convert to TFLite with FP16 Quantization
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.target_spec.supported_types = [tf.float16]

    tflite_model = converter.convert()

    # 3. Save
    tflite_path = f'/content/drive/MyDrive/Research_Project/Code/models/fusion_resnet_fold{fold_number}_fp16.tflite'
    with open(tflite_path, 'wb') as f:
        f.write(tflite_model)

    # 4. Compare Sizes
    keras_size = os.path.getsize(model_path) / 1024
    tflite_size = os.path.getsize(tflite_path) / 1024
    print(f"Fold {fold_number} - Keras: {keras_size:.2f} KB | TFLite: {tflite_size:.2f} KB | Reduction: {(1-tflite_size/keras_size)*100:.1f}%")

# Quantize the best fold (Fold 5 based on your logs)
print("--- QUANTIZING FOR EDGE DEPLOYMENT ---")
quantize_model(5)

In [ ]:
# Install pyngrok to expose the local server
!pip install streamlit pyngrok

In [ ]:
# Kill existing ngrok and streamlit processes
!pkill ngrok
!pkill streamlit


In [ ]:
from pyngrok import ngrok

# Authenticate ngrok (Optional but recommended for stability)
# Sign up at ngrok.com for a free token if you want the link to last longer
!ngrok authtoken "paste_your_token_here"

# Run Streamlit in the background
!streamlit run /content/drive/MyDrive/Research_Project/Code/app.py &>/dev/null&

# Create the public tunnel
public_url = ngrok.connect(addr='8501')
print(f"🚀 Dashboard Live at: {public_url}")